In [73]:
# using t-stages instead of a time series dataframe to track combat for reinforcement learning
import numpy as np
import pandas as pd
import random
import math
import time
import matplotlib.pyplot as plt
from matplotlib import style

In [74]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

In [75]:
import pygame

In [76]:
import Species
import Backgrounds
import Armor_and_Weapons
import Dice_Rolls
import Establishing_Hierarchy
import CHARACTER_CREATOR
import Monsters
import Character_Actions

### Creating Environment

In [77]:
def generate_situation(character_num):
    print('generating situation...')

    # for the number of characters, randomly generate a Player_Character using Random_Character()
    entities = []
    print('generating characters...')
    for i in range(character_num):
        entities.append(CHARACTER_CREATOR.Random_Character())

    # for the number of entities, within each entity's flesh out the Active_Conditions and Circumstances dictionaries
    for entity in range(len(entities)):
        other_entities = entities.copy()
        other_entities.pop(entity)
        
        self_name = str(entities[entity].Name)
        entities[entity].Circumstances['Attack Rolls'][self_name] = None
        # I want to add a key for each other entity in the Active_Conditions dictionaries
        for other_entity in range(len(other_entities)):
            entities[entity].Active_Conditions[str(entities[other_entity])] = []
            other_entity_name = str(other_entities[other_entity].Name)
            entities[entity].Circumstances['Attack Rolls'][other_entity_name] = None

            #entities[entity].Circumstances['Attack Rolls'][entity_name] = 'placeholder'
            #print(entities[other_entity].Name)
    return entities

In [78]:

def generate_combat(character_num):
    print('generating characters...')
    combat_situation = generate_situation(character_num)
    

    print('adding actions to characters...')
    for entity in range(len(combat_situation)):
    #    combat_situation[entity].Actions = [Character_Actions.No_Action(combat_situation[entity],combat_situation)]
    #    combat_situation[entity].Bonus_Actions = combat_situation[entity].Bonus_Actions.append(Character_Actions.No_Bonus_Action(combat_situation[entity],combat_situation,))
    #    combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.Attack_Action(combat_situation[entity],combat_situation))
    #    combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.Dodge_Action(combat_situation[entity],combat_situation))
    #    combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.Help_Action(combat_situation[entity],combat_situation))
    #    combat_situation[entity].Free_Actions = combat_situation[entity].Free_Actions.append(Character_Actions.Move(combat_situation[entity],combat_situation))
    #    Character_Actions.Equip_Weapon(combat_situation[entity],Character_Actions.Choose_Random_Weapon(combat_situation[entity]))

        # assign a unique location to each entity within a 12x12 grid
        combat_situation[entity].Location['X'] = np.random.randint(0,12)
        combat_situation[entity].Location['Y'] = np.random.randint(0,12)
        combat_situation[entity].Location['Z'] = 0

    print('building environment...')
    class CombatEnv(Env):
        def __init__(self):
            # the action space needs to combine Action, Bonus Action, and Movement
            # so the action space might need to be a dictionary with keys "Action", "Bonus Action", and "Movement"
            # and the values will be the actions that can be taken


            # I need to generate creatures first and foremost
            # and then for each action they are capable of, assign action spaces

            # action_space_dict = Dict({
            #    "Action": Discrete(4),
            #    'Action Order': Discrete(3), # 3 = Action goes last, 2 = Action goes second, 1 = Action goes first
            #    "Bonus Action": Discrete(4),
            #    'Bonus Action Order': Discrete(3), # 3 = Bonus Action goes last, 2 = Bonus Action goes second, 1 = Bonus Action goes first
            #    "Movement": Discrete(4), # how tf am I going to implement split movement
            #})
            # and then a function to convert it into something usable

            #for character in combat_situation:
            #    character_action_space = spaces.Discrete(len(character.actions))


            ##### State Array
            state_array = []
            for character in combat_situation:
                state_array.append(character.Name + ' Current_HP')
                state_array.append(character.Name + ' Temp_HP')
                state_array.append(character.Name + ' Size')
                state_array.append(character.Name + ' Walking Speed')
                state_array.append(character.Name + ' Flying Speed')
                state_array.append(character.Name + ' Str_Score')
                state_array.append(character.Name + ' Dex_Score')
                state_array.append(character.Name + ' Con_Score')
                state_array.append(character.Name + ' Int_Score')
                state_array.append(character.Name + ' Wis_Score')
                state_array.append(character.Name + ' Cha_Score')
                state_array.append(character.Name + ' Active_Conditions')
                state_array.append(character.Name + ' Concentrating')
                state_array.append(character.Name + 'Location X')
                state_array.append(character.Name + 'Location Y')
                state_array.append(character.Name + 'Location Z')
            
            self.state = np.array(state_array)

            for entity in range(len(combat_situation)):
                combat_situation[entity].Bonus_Actions = []
                combat_situation[entity].Bonus_Actions = combat_situation[entity].Bonus_Actions.append(Character_Actions.No_Bonus_Action(combat_situation[entity],combat_situation,self.state))
                
                
                print(combat_situation[entity])
                print(combat_situation[entity].Actions)
                combat_situation[entity].Actions = []
                print(Character_Actions.No_Action(combat_situation[entity],combat_situation,self.state))
                combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.No_Action(combat_situation[entity],combat_situation,self.state))
                print(combat_situation[entity].Actions)

                combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.Attack_Action(combat_situation[entity],combat_situation,self.state))
                combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.Dodge_Action(combat_situation[entity],combat_situation,self.state))
                combat_situation[entity].Actions = combat_situation[entity].Actions.append(Character_Actions.Help_Action(combat_situation[entity],combat_situation,self.state))
                combat_situation[entity].Free_Actions = combat_situation[entity].Free_Actions.append(Character_Actions.Move(combat_situation[entity],combat_situation,self.state))
                Character_Actions.Equip_Weapon(combat_situation[entity],Character_Actions.Choose_Random_Weapon(combat_situation[entity]))

                # assign a unique location to each entity within a 12x12 grid
                combat_situation[entity].Location['X'] = np.random.randint(0,12)
                combat_situation[entity].Location['Y'] = np.random.randint(0,12)



            ###### Observation Space
            # do I need an observation space per character?
            self.observation_space = Box(low=0, high=1, shape=(len(self.state),), dtype=np.float32)


            ###### Action Space
            for character in combat_situation:
                action_space_name = str(combat_situation[character].Name) + '_action_space'
                action_space_name = gym.spaces.Discrete(len(combat_situation[character].Actions))

            ###### Other
            self.max_turns = 10
            
        def step(self, action):
            pass

        def render(self):
            pass

        def reset(self):
            pass
        
            



    def take_turn(combat_log_new,character):
        # the take_turns function needs to be overhauled so that it's attaching only one row at a time to the combat_log_new dataframe
        # that means that it needs to choose Action, Bonus_Action, or Movement, then enact it, then concat it to the combat_log_new dataframe before continuing with the rest
        # that new last row will be the basis for the next action




        # need to decide which action_type to take first: Actions or Bonus_Actions
        action_types = ['Actions','Bonus_Actions','Movement']
            # Move Splitting
            # Free Action
            # Object Interaction
        

    
    # making a round 0 to represent the spawning conditions
    first_row = pd.DataFrame({'Combat Round':0,
                                                              'Action Time': None,
                                                              'Action Name': None, 
                                                              'Action Type': None,
                                                              'Target': None,
                                                              'Action Result': None,
                                                              'Current Allied Ability Check':'Spawning','Current Allied Attack Roll':'Spawning','Current Allied Saving Throw':'Spawning','Current Allied Damage Roll':'Spawning','Current Enemy Ability Check':'Spawning','Current Enemy Attack Roll':'Spawning','Current Enemy Saving Throw':'Spawning','Current Enemy Damage Roll':'Spawning'}, index=[0])

    for entity in range(len(combat_situation)):
        first_row[combat_situation[entity].Name + ' Acting True'] = 0
        first_row[combat_situation[entity].Name + ' Current_HP'] = combat_situation[entity].HP
        first_row[combat_situation[entity].Name + ' Temp_HP'] = combat_situation[entity].Temp_HP
        first_row[combat_situation[entity].Name + ' Size'] = combat_situation[entity].Size
        first_row[combat_situation[entity].Name + ' Walking Speed'] = combat_situation[entity].Speed['Walking']
        first_row[combat_situation[entity].Name + ' Flying Speed'] = combat_situation[entity].Speed['Flying']
        first_row[combat_situation[entity].Name + ' Str_Score'] = combat_situation[entity].Str_Score
        first_row[combat_situation[entity].Name + ' Dex_Score'] = combat_situation[entity].Dex_Score
        first_row[combat_situation[entity].Name + ' Con_Score'] = combat_situation[entity].Con_Score
        first_row[combat_situation[entity].Name + ' Int_Score'] = combat_situation[entity].Int_Score
        first_row[combat_situation[entity].Name + ' Wis_Score'] = combat_situation[entity].Wis_Score
        first_row[combat_situation[entity].Name + ' Cha_Score'] = combat_situation[entity].Cha_Score
        first_row[combat_situation[entity].Name + ' Active_Conditions'] = combat_situation[entity].Active_Conditions
        first_row[combat_situation[entity].Name + ' Concentrating'] = combat_situation[entity].Concentrating
        first_row[combat_situation[entity].Name + 'Location X'] = combat_situation[entity].Location['X']
        first_row[combat_situation[entity].Name + 'Location Y'] = combat_situation[entity].Location['Y']
        first_row[combat_situation[entity].Name + 'Location Z'] = combat_situation[entity].Location['Z']

    # attach the new_round dictionary to the combat_log_new dataframe using concat
    #combat_log_new = pd.concat([combat_log_new,pd.DataFrame(first_row)],ignore_index=False)

    #for character in range(len(combat_situation)):
        # print their starting health
    #    print('Character: ' + combat_situation[character].Name + ' has ' + str(combat_situation[character].Current_HP) + ' HP')

    #print('taking turns...')
    #for Combat_Round in range(1,3):
    #    combat_round += 1

    #    for entity in range(len(combat_situation)):
    #        combat_log_new = pd.concat([combat_log_new,take_turn(combat_log_new,entity)], ignore_index=True, axis=0)


    #combat_log_new = combat_log_new.reindex(columns=['Combat Round'] + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Acting True')]) + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Current_HP')]) + list(combat_log_new.columns[1:][~combat_log_new.columns[1:].str.contains('Acting True') & ~combat_log_new.columns[1:].str.contains('Current_HP')]))
    
    return CombatEnv()

In [79]:
'''
class CombatEnv(Env):
    def __init__(self):
        # the action space needs to combine Action, Bonus Action, and Movement
        # so the action space might need to be a dictionary with keys "Action", "Bonus Action", and "Movement"
        # and the values will be the actions that can be taken


        # I need to generate creatures first and foremost
        # and then for each action they are capable of, assign action spaces

        # action_space_dict = Dict({
        #    "Action": Discrete(4),
        #    'Action Order': Discrete(3), # 3 = Action goes last, 2 = Action goes second, 1 = Action goes first
        #    "Bonus Action": Discrete(4),
        #    'Bonus Action Order': Discrete(3), # 3 = Bonus Action goes last, 2 = Bonus Action goes second, 1 = Bonus Action goes first
        #    "Movement": Discrete(4), # how tf am I going to implement split movement
        #})
        # and then a function to convert it into something usable

        #for character in combat_situation:
        #    character_action_space = spaces.Discrete(len(character.actions))



        # the combat array
        self.observation_space = Box(low=0, high=100, shape=(3,))
        
        ######## creating a state
        # what needs to be represented here?
        # for each creature in environment: Maximum_HP, Current_HP, Temp_HP, AC, Size, Position, Speed, Strength, Dexterity, Constitution, Intelligence, Wisdom, Charisma, Active Conditions, Concentrating,  

        self.state = [100, 100, 100] # placeholder for now
        
        # set episode length
        self.max_turns = 10

    def step(self, action):
        # apply actions
        self.state += action
        if self.state[0] > 100:
            reward = 1
        else:
            reward = 0
            
        if placeholder <= 0:
            done = True
        else:
            done = False
        info = {}
        return self.state, reward, done, info
        
    def render(self):
        pass

    def reset(self):
        self.state = [100, 100, 100]
        return self.state
'''

'\nclass CombatEnv(Env):\n    def __init__(self):\n        # the action space needs to combine Action, Bonus Action, and Movement\n        # so the action space might need to be a dictionary with keys "Action", "Bonus Action", and "Movement"\n        # and the values will be the actions that can be taken\n\n\n        # I need to generate creatures first and foremost\n        # and then for each action they are capable of, assign action spaces\n\n        # action_space_dict = Dict({\n        #    "Action": Discrete(4),\n        #    \'Action Order\': Discrete(3), # 3 = Action goes last, 2 = Action goes second, 1 = Action goes first\n        #    "Bonus Action": Discrete(4),\n        #    \'Bonus Action Order\': Discrete(3), # 3 = Bonus Action goes last, 2 = Bonus Action goes second, 1 = Bonus Action goes first\n        #    "Movement": Discrete(4), # how tf am I going to implement split movement\n        #})\n        # and then a function to convert it into something usable\n\n        #

In [80]:
env = generate_combat(4)
states = env.observation_space.shape[0]
actions = env.action_space.n

generating characters...
generating situation...
generating characters...
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Create Character Function - Class: <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Create Character Function - Class: <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Create Character Function - Class: <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Character_Creator - Random Charact

AttributeError: 'NoneType' object has no attribute 'append'

In [ ]:
actions

4

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))
env.close()

TypeError: 'int' object is not iterable

### Creating Model

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

NameError: name 'env' is not defined

In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

### Building Agent

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory